In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os
import pandas as pd
import scanpy as sc
import cellink as cl

/data/nasif12/home_if12/l_mai/.conda/envs/mm_scgenetics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from cellink.tl._burden_testing import *

In [5]:
cell_type_col = "cell_label"

In [3]:
base_data_dir = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/")
input_dir = base_data_dir / "input_data"
output_dir = base_data_dir / "output"
## defining the path to the data
scdata_path = input_dir / "chr22_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz"
gdata_path = input_dir / "plink/"
dump_path = base_data_dir / "output/example_output"
vep_anno_file = input_dir /  "all_variants_annotated_fixed_ref_reversed_id.txt"
DNA_LM_file = input_dir/ "annotations/onek1k_inf_scores_upstream_model.tsv"
## defining target chromosome
target_chromosome = "22"

In [7]:
# data
data = pd.read_pickle(output_dir/"data_preprocessed.pkl")

In [9]:
data.gdata.varm["annotations_0"]

gnomADe_EAS_AF Feature_type  Consequence_intergenic_variant  \
snp_id                                                                         
22_16849573_G_A             NaN            -                               1   
22_16849971_T_A             NaN            -                               1   
22_16850437_A_G             NaN            -                               1   
22_16851225_T_C             NaN            -                               1   
22_16851356_T_C             NaN            -                               1   
...                         ...          ...                             ...   
22_51202748_G_A             NaN   Transcript                               0   
22_51208568_T_G             NaN   Transcript                               0   
22_51211031_G_A             NaN   Transcript                               0   
22_51213613_T_C             NaN   Transcript                               0   
22_51216564_C_T             NaN   Transcript                               0   

                PHENO  Consequence_start_lost  gnomADe_AMR_AF  \
snp_id                                                          
22_16849573_G_A     -                       0             NaN   
22_16849971_T_A     -                       0             NaN   
22_16850437_A_G     -                       0             NaN   
22_16851225_T_C     -                       0             NaN   
22_16851356_T_C     -                       0             NaN   
...               ...                     ...             ...   
22_51202748_G_A     -                       0             NaN   
22_51208568_T_G     -                       0             NaN   
22_51211031_G_A     -                       0             NaN   
22_51213613_T_C     -                       0             NaN   
22_51216564_C_T     -                       0             NaN   

                Existing_variation CLIN_SIG  CADD_RAW  gnomADe_NFE_AF  ...  \
snp_id                                                                 ...   
22_16849573_G_A        rs112435201        -  0.433139             NaN  ...   
22_16849971_T_A          rs7287956        -  0.442607             NaN  ...   
22_16850437_A_G          rs5748209        -  0.369731             NaN  ...   
22_16851225_T_C          rs5746874        -  0.393139             NaN  ...   
22_16851356_T_C          rs5748581        -  0.377289             NaN  ...   
...                            ...      ...       ...             ...  ...   
22_51202748_G_A          rs9616963        -  0.550962             NaN  ...   
22_51208568_T_G        rs148425445        -  0.102244             NaN  ...   
22_51211031_G_A          rs9616968        -  0.153297             NaN  ...   
22_51213613_T_C         rs34726907        - -0.394121             NaN  ...   
22_51216564_C_T          rs9616970        - -0.113869             NaN  ...   

                 Consequence_splice_donor_5th_base_variant  \
snp_id                                                       
22_16849573_G_A                                          0   
22_16849971_T_A                                          0   
22_16850437_A_G                                          0   
22_16851225_T_C                                          0   
22_16851356_T_C                                          0   
...                                                    ...   
22_51202748_G_A                                          0   
22_51208568_T_G                                          0   
22_51211031_G_A                                          0   
22_51213613_T_C                                          0   
22_51216564_C_T                                          0   

                 Consequence_splice_donor_region_variant  \
snp_id                                                     
22_16849573_G_A                                        0   
22_16849971_T_A                                        0   
22_16850437_A_G                                        0   
22_16851

In [ ]:
data.gdata.varm["annotations_0"]

### Understanding burden computation

In [32]:
this_vars = data.gdata.varm["annotations_0"][data.gdata.varm["annotations_0"]["Gene"] == "ENSG00000184319"].index
this_vars


Index(['22_51197576_C_G', '22_51198868_C_T', '22_51198906_A_G',
       '22_51198998_A_C', '22_51202748_G_A', '22_51208568_T_G',
       '22_51211031_G_A', '22_51213613_T_C', '22_51216564_C_T'],
      dtype='object', name='snp_id')

In [24]:
data.gdata.varm["annotations_0"]["Gene"]

snp_id
22_16849573_G_A                  -
22_16849971_T_A                  -
22_16850437_A_G                  -
22_16851225_T_C                  -
22_16851356_T_C                  -
                        ...       
22_51202748_G_A    ENSG00000184319
22_51208568_T_G    ENSG00000184319
22_51211031_G_A    ENSG00000184319
22_51213613_T_C    ENSG00000184319
22_51216564_C_T    ENSG00000184319
Name: Gene, Length: 96257, dtype: object

In [28]:
data.gdata.varm["annotations_0"]["Location"]

snp_id
22_16849573_G_A    22:16849573
22_16849971_T_A    22:16849971
22_16850437_A_G    22:16850437
22_16851225_T_C    22:16851225
22_16851356_T_C    22:16851356
                      ...     
22_51202748_G_A    22:51202748
22_51208568_T_G    22:51208568
22_51211031_G_A    22:51211031
22_51213613_T_C    22:51213613
22_51216564_C_T    22:51216564
Name: Location, Length: 96257, dtype: object

In [46]:
## we need the location of the gene-> choose vars based on window
#todo: test +/- strand

In [17]:
import requests

def get_gene_location(ensembl_id):
    url = f"https://rest.ensembl.org/lookup/id/{ensembl_id}?content-type=application/json"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        #print(data)
        if data.get('strand') == 1: # forward
            return f"{data.get('seq_region_name')}:{data.get('start')}-{data.get('end')}"
        else: # reverse
            return f"{data.get('seq_region_name')}:{data.get('end')}-{data.get('start')}"
    else:
        return f"Error: {response.status_code}, {response.text}"


In [20]:
#eg: forward
gene_id = "ENSG00000184319"
location = get_gene_location(gene_id)
print(location)

#eg: rev
gene_id = "ENSG00000118137"
location = get_gene_location(gene_id)
print(location)


22:50798655-50799123
11:116837622-116835751


# Check results of burden testing
with window size 100k

In [4]:
res = pd.read_pickle(output_dir/"all_results_DNA_LM_and_MAF_100k.pkl")


In [5]:
res

DISTANCE  CADD_PHRED  DNA_LM_influence_score  MAF_beta_1.25  \
id                                                                       
1_1             0.0       0.000                0.000000       0.000000   
2_2             0.0       0.000                0.000000       0.000000   
3_3             0.0       0.000                0.000000       0.000000   
4_4             0.0       0.000                0.000000       0.000000   
6_6             0.0       0.000                0.000000       0.000000   
...             ...         ...                     ...            ...   
1077_1078   13629.0      61.807                0.342296     186.380891   
1078_1079   54694.0     330.785                1.901447    1011.361912   
1079_1080   11470.0      50.978                0.202178     118.862321   
1080_1081   10698.0      45.424                0.155111      98.863939   
1081_1082   10019.0      42.980                0.185957     130.097863   

                    Geneid  
id                          
1_1        ENSG00000198445  
2_2        ENSG00000198445  
3_3        ENSG00000198445  
4_4        ENSG00000198445  
6_6        ENSG00000198445  
...                    ...  
1077_1078  ENSG00000079974  
1078_1079  ENSG00000079974  
1079_1080  ENSG00000079974  
1080_1081  ENSG00000079974  
1081_1082  ENSG00000079974  

[660213 rows x 5 columns]

In [18]:
res.keys()

Index(['DISTANCE', 'CADD_PHRED', 'DNA_LM_influence_score', 'MAF_beta_1.25',
       'Geneid'],
      dtype='object')

In [19]:
# all unique geneIds
unique_geneids = res['Geneid'].unique()
print(f"Total GeneIds: {str(len(unique_geneids))}")
zero_geneids = res[(res['DISTANCE'] == 0.0) & (res['CADD_PHRED'] == 0.0) & (res['DNA_LM_influence_score'] == 0.0) & (res['MAF_beta_1.25'] == 0.0)]['Geneid'].unique()
print(f"zero value GeneIds: {str(len(zero_geneids))}")

Total GeneIds: 673
zero value GeneIds: 256


In [27]:
# these are all genes where the ensembl id did not return a gene location
missing_gene = []
with open("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/missing_genes_for_burden_calc.txt", "r") as file:
    for line in file:
        line = line.strip()
        if line not in missing_gene:
            missing_gene.append(line)

print(len(missing_gene))

35


In [28]:
missing_gene[:10]

['ENSG00000273203',
 'ENSG00000272675',
 'ENSG00000188424',
 'ENSG00000223579',
 'ENSG00000234252',
 'ENSG00000225070',
 'ENSG00000198477',
 'ENSG00000270041',
 'ENSG00000272955',
 'ENSG00000260065']

In [30]:
# check how many zero value geneId that are not in missing genes
missing = [item for item in zero_geneids if item not in missing_gene]
len(missing)

225

In [31]:
# check how many zero value geneId that are  in missing genes
overlap = [item for item in zero_geneids if item in missing_gene]
len(overlap)

31

In [32]:
filtered_res = res[~res['Geneid'].isin(missing_gene)]
filtered_res

DISTANCE  CADD_PHRED  DNA_LM_influence_score  MAF_beta_1.25  \
id                                                                       
1_1             0.0       0.000                0.000000       0.000000   
2_2             0.0       0.000                0.000000       0.000000   
3_3             0.0       0.000                0.000000       0.000000   
4_4             0.0       0.000                0.000000       0.000000   
6_6             0.0       0.000                0.000000       0.000000   
...             ...         ...                     ...            ...   
1077_1078   13629.0      61.807                0.342296     186.380891   
1078_1079   54694.0     330.785                1.901447    1011.361912   
1079_1080   11470.0      50.978                0.202178     118.862321   
1080_1081   10698.0      45.424                0.155111      98.863939   
1081_1082   10019.0      42.980                0.185957     130.097863   

                    Geneid  
id                          
1_1        ENSG00000198445  
2_2        ENSG00000198445  
3_3        ENSG00000198445  
4_4        ENSG00000198445  
6_6        ENSG00000198445  
...                    ...  
1077_1078  ENSG00000079974  
1078_1079  ENSG00000079974  
1079_1080  ENSG00000079974  
1080_1081  ENSG00000079974  
1081_1082  ENSG00000079974  

[625878 rows x 5 columns]

In [33]:
print(len(res.index))
print(len(filtered_res.index))

660213
625878


In [33]:
# Save data object
import pickle 
with open(output_dir/"all_results_DNA_LM_and_MAF_100k_filtered.pkl", "wb") as f:
    pickle.dump(filtered_res, f)

print("AnnData object saved to all_results_DNA_LM_and_MAF_100k_filtered.pkl")

AnnData object saved to all_results_DNA_LM_and_MAF_100k_filtered.pkl


## check if ensembl ids of missing locations have something in common

In [5]:
import requests

In [12]:
def fetch_gene_info(ensembl_id):
    # Base URL for Ensembl REST API
    base_url = "https://rest.ensembl.org/lookup/id/"
    # Send a GET request to fetch gene information
    response = requests.get(f"{base_url}{ensembl_id}", params={"content-type": "application/json"})

    # Check if the gene exists and if it is deprecated
    print("got response")
    if response.status_code == 200:
        data = response.json()

        if "status" in data and data["status"] == "deprecated":
            print(f"Gene {ensembl_id} is deprecated.")
        else:
            print(f"Gene {ensembl_id} is valid.")
        
        return data
    else:
        print(f"Gene {ensembl_id} not found.")
        return None


In [17]:
def get_ensembl_id_info(ensembl_id):
    url = f"https://rest.ensembl.org/lookup/id/{ensembl_id}?expand=1"
    headers = {"Content-Type": "application/json"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()  # Returns ID information
    elif response.status_code == 404:
        return {"error": "ID not found. It may be deprecated or invalid."}
    elif response.status_code == 400:
        return {"error": "Bad Request. Check the ID format or URL structure."}
    else:
        return {"error": f"Unexpected error: {response.status_code}"}

# Example usage
ensembl_id = "ENSG00000273203"
info = get_ensembl_id_info(ensembl_id)
print(info)

{'error': 'Bad Request. Check the ID format or URL structure.'}


In [14]:
gene_info = fetch_gene_info("ENSG00000273203")

if gene_info:
    print("Gene Information:", gene_info)


got response
Gene ENSG00000273203 not found.


In [16]:
gene_info = get_ensembl_id_info("ENSG00000273203")

if gene_info:
    print("Gene Information:", gene_info)


Gene Information: {'error': 'Unexpected error: 400'}
